# My DataBase of Favourite Movies
Here is how to create a database including DB design and a simple python implementation. We will use SQLite3 that comes with Python and while being limited for small DB projects, it's an immediate way to get familiar with the SQL language common to all relational DB management softwares. 

As an example here, we will create a database to manage the metadata of my favourite movies, including entities such as actors, directors and awards. The goal is to store all this information and be able to ask questions like: which of my favourite movies are by Quentin Tarantino? or which are the best rated movies with Geoffrey Rush I ever watched? 

## 1. Entity Relational Diagram (ERD) design
First of all we need to design the architecture of our DB. The easiest way is to design the Entity Relation Diagram (ERD). This can be done online using ERDplus.com. 

*(Although learning to design a ERD might seem cumbersome and unnecessary at the beginning, it become more and more necessary as the DB becomes more complex, and it will actually speed up the SQL coding as we will see in section 3).*

<img src="erd_cinema.png" >

The ERD diagram represents the logical structure of the information to be stored. Although it looks jumbly at first, it's logic can be easily grasped, for example the "Actors" are connected to "Movies" because they "starred in" them, while they have no direct connection to "Directors" uless through the "Movies" that where "Directed by" them. 

The basic idea of a ERD is to break down the project into:
1. **Entities** (in box e.g. movies, actors) which are self-contained logical entities
2. **Relationships** (in diamonds e.g. directed by, movie award) 
3. **Attributes** are attached to entities or relationships (in oval e.g. the title, year, actor's name)
  * **Primary keys** (bold underlined e.g. actor_ID) are the most common example of *unique identifier* and are necessary elements of a relational DB. They allow to ensure data-integrity and to eventually create the relashioships between different entities in the DB
  * **Multivalue** attributes (in double oval e.g. a movie can have multiple producers, or rated by different institutions)

The connecting lines between entities are part of the relationship and the symbols on either ends is called **cardinality** and depicts for example a one-to-one, one-to-many or many-to-many relationship. More details about the ERD notation can be found for example here https://www.lucidchart.com/pages/ER-diagram-symbols-and-meaning.

## 2. Relational Schema
While the ERD is a logical representation of the project, a more direct way to represent how the DB should be implemented is the Relational Schema. In the schema, the "actual" tables are depicted, including those for both Entities and Relationships, with more information about how to connect the entities using Keys.

Usually the step from ERD to the Schema is mecchanical and ERDPlus.com offers a tool to convert the ERD to a Schema. And here is what we obtain:

<img src="schema_cinema.png">

Here the same structure as the ERD can be recognized with some differences:
1. Relashioships become a table with two **Foreign Keys (FK)** unless they are optional relationships (e.g. not every actor or movie has an Award)
2. Multivalued attributes become their own tables to respect the principle of Atomicity of the DB (for more info on DB design principles look up the three Normalization Rules)

Now we are ready to implement our design into an actual DB using SQLite3.

## 3. Create the SQLite3 Database
Now we can use sqlite3 to create the db file and open the connection:

In [20]:
import sqlite3

# this will create the .db file if it doesn't exist, and connect to it
conn = sqlite3.connect('cinema.db', timeout = 10) 
# we define our cursor
c = conn.cursor()

A file called cinema.db should have appeared in your working directory. This file will contain the whole database in binary format. To opened and explored the .db file, download the "DB Browser for SQLite", that can be found at: https://github.com/sqlitebrowser/sqlitebrowser/releases

In [19]:
# Remember to close the cursor and connection when you are done with:
c.close()
conn.close()

Now its time to populate the DB with tables and columns. This requires knowledge of SQL language and will be explained as we go.

Here is how we create the first table, syntax words are in CAPS which definitions of tables and columns is in lower-case:

In [21]:
c.execute('CREATE TABLE IF NOT EXISTS Movies     \
            (movie_ID INTEGER PRIMARY KEY,       \
            title_en VARCHAR,                    \
            title_IT VARCHAR,                    \
            movie_year INT,                      \
            watched INT                          \
            );')

The basic syntax to create tables is:

**CREATE TABLE table_name (attribute1, attribute2,...)**

Notice that every attribute is followed by the DataType (e.g. INT, VARCHAR) and at the end the PRIMARY KEY is also identified. Similarly we can create the other main tables:

In [22]:
c.execute('\
    CREATE TABLE Directors            \
    (director_ID INTEGER PRIMARY KEY, \
    director_Fname VARCHAR NOT NULL,  \
    director_Lname VARCHAR NOT NULL,  \
    direc_birthYear INT               \
    );')
c.execute('\
    CREATE TABLE Actors               \
    (actor_ID INTEGER PRIMARY KEY,    \
    actor_Fname VARCHAR NOT NULL,     \
    actor_Lname VARCHAR NOT NULL,     \
    actor_birthYear INT               \
    );')

The rest of the tables, those referrring to entity-relationships and the table Awards are implemented with the only difference that they include *Foreign Keys* which are defined with an intuitive command syntax (see commands below).

In [23]:
c.execute("                                               \
    CREATE TABLE Awards                                   \
    (award_ID INTEGER PRIMARY KEY,                        \
    Oscar_year INT,                                       \
    Oscar_category VARCHAR,                               \
    OtherAward1_year INT,                                 \
    Highlight VARCHAR,                                    \
    actor_ID INT,                                         \
    movie_ID INT,                                         \
    FOREIGN KEY (actor_ID) REFERENCES Actors(actor_ID),   \
    FOREIGN KEY (movie_ID) REFERENCES Movies(movie_ID)    \
    );")

c.execute("                                                       \
    CREATE TABLE directed_by                                      \
    (director_ID INT NOT NULL,                                    \
    movie_ID INT NOT NULL,                                        \
    PRIMARY KEY (director_ID, movie_ID),                          \
    FOREIGN KEY (director_ID) REFERENCES Directors(director_ID),  \
    FOREIGN KEY (movie_ID) REFERENCES Movies(movie_ID)            \
    );")

c.execute("                                               \
    CREATE TABLE starred_in                               \
    (actor_ID INT NOT NULL,                               \
    movie_ID INT NOT NULL,                                \
    PRIMARY KEY (actor_ID, movie_ID),                     \
    FOREIGN KEY (actor_ID) REFERENCES Actors(actor_ID),   \
    FOREIGN KEY (movie_ID) REFERENCES Movies(movie_ID)    \
    );")                                                        #   

c.execute("                                               \
    CREATE TABLE Movies_producers                         \
    (producer_name VARCHAR NOT NULL,                      \
    movie_ID INT NOT NULL,                                \
    PRIMARY KEY (producer_name, movie_ID),                \
    FOREIGN KEY (movie_ID) REFERENCES Movies(movie_ID)    \
    );")

c.execute('                                               \
    CREATE TABLE Movies_ratings                           \
    (ratings_ID INT NOT NULL,                             \
    Rotten_Tomatoes INT NOT NULL,                         \
    movie_ID INT NOT NULL,                                \
    PRIMARY KEY (ratings_ID, movie_ID),                   \
    FOREIGN KEY (movie_ID) REFERENCES Movies(movie_ID)    \
    );')

### ...IMPORTANT HINT:
An easy way to write all this SQL code, which can be borign to do manually, is to Export the SQL code from ERDPlus.com once the Relational Schema is designed. This is a very useful free tool that save a lot of time and tedious, carefull coding, and it's a strong argument for designing the ERD with a suitable software in the first place.

## 4. Data population

Before you populate the database make sure you are still connected to the database, to reconnect you can use the following commands:

In [60]:
conn = sqlite3.connect('cinema.db', timeout = 10) 
c = conn.cursor()

Before you insert data in a table, you might want to check what are the available columns and their names. You can explore the information of a table (e.g. "Movies") with the following commands. Note that every time you try and get data from the DB, you need to send the query and then fetchall() selected data.

In [24]:
c.execute('PRAGMA table_info(Movies);') #
data = c.fetchall()
for row in data:
    print(row)

(0, 'movie_ID', 'INTEGER', 0, None, 1)
(1, 'title_en', 'VARCHAR', 0, None, 0)
(2, 'title_IT', 'VARCHAR', 0, None, 0)
(3, 'movie_year', 'INT', 0, None, 0)
(4, 'watched', 'INT', 0, None, 0)


Now you can insert your first "row" into one of the tables with the following command:

**INSERT INTO table_name (column1, column2,...)  
VALUES (value1, value2, ...)**

In [81]:
c.execute("INSERT INTO Movies (title_en, movie_year) VALUES ('the daughter', 2015);")
conn.commit()

Notice that after every INSERT, data have to be committed for the insertion to take effect. Also notice that if the primary key is defined as an INTEGER, then it can be omitted in the INSERT query and an automatic incremental value will be assigned.

In [49]:
c.execute("INSERT INTO Movies (title_en, movie_year, watched, genre) \
          VALUES ('the circle', 2017, 1, 'sci-fi'),                  \
                 ('the beauty and the beast', 2017, 1, 'fantasy'),   \
                 ('interstellar', 2014, 1, 'sci-fi'),                \
                 ('the martian', 2015, 1, 'sci-fi')                  \
          ;")
conn.commit()   # insert multiple movies with a single query

Let's insert some actors as well and then see how we can link tables with Foreign Keys:

In [51]:
c.execute("INSERT INTO Actors (actor_Fname, actor_Lname, actor_birthYear)  \
          VALUES ('Emma', 'Watson', 1990),                                 \
                 ('Matt', 'Damon', 1970)                                   \
          ;")
conn.commit()

Now we are ready to insert data into the relationship tables, and since this tables contain foreign keys we need to make sure we retrieve them correctly by SELECTing those keys from their respective tables. In this case we will use the WHERE condition to identify them. 

In [54]:
c.execute("INSERT INTO starred_in (actor_ID, movie_ID)   \
          SELECT actor_ID, movie_ID                      \
          FROM Actors, Movies                            \
          WHERE actor_Fname = 'Matt'                     \
          AND title_en = 'the martian';     \
          ")
conn.commit()

Note that it would have been much easier to insert in this table (and all reletionship tables) if the PRIMARY KEY were a value we know, like the actor name or the movie title. Nevertheless, we would have to know it correctly and make sure that a movie title is never ambiguous (otherwise cannot work as PK, because it is not a unique identifier). In this situation is often wiser to use **SURROGATE KEYS** i.e. incremental integers that act as KEY, although making the queries more nested to write.

## 5. Querying data

Now we are ready to start querying data and reap the results of our work. Even if the DB only contains few entries, it is easy to imagine the power of this approach when its size is scaled up by many order of magnitude.

The main command to learn to retrieve data is:

** SELECT column1, column2, ... FROM table_name**

In [55]:
c.execute("SELECT actor_Fname, actor_Lname FROM Actors;")
data = c.fetchall()
for row in data: print(row)

('Emma', 'Watson')
('Matt', 'Damon')


Notice that again data have to fetched to be printed. 

The syntax of the SELECT command is very rich and here is the sckeleton:

**SELECT column  
FROM table_name  
WHERE condiiton  
GROUP BY column  
ORDER BY column  
LIMIT number**

Every column and table can be calles with an ALIAS (**SELECT column1 AS c1**) to lighten the query. The WHERE supports boolean logic like AND, OR, IS, IS NOT, LIKE regex..., and also GROUP BY and ORDER BY can have multiple levels, and finally LIMIT supports both an off-set and a range. 

So as you see the syntax is rich and allows for sofisticated analysis as other operators are introduced, such as SUM(), COUNT(), MAX(), AVERAGE(), ecc.

Here is an example of more complex query where we look for actors older than me whos first name starts with the letter "m" (SQL is not case sensitive!!).

In [56]:
c.execute("SELECT *                      \
          FROM Actors                    \
          WHERE actor_birthYear < 1986   \
          AND actor_Fname LIKE 'm%';     \
          " ) 
data = c.fetchall()
for row in data: print(row)

(2, 'Matt', 'Damon', 1970)


## 5. Querying across multiple tables with "JOIN"

The "JOIN" is a very important command for more advanced querying. It allows to really exploit the potential of relational databases by looking for specific subsets of data spread across multiple tables while keeping the process computationally efficient. 

There are different kinds of JOIN-ing operations that correspond to different set-theory equivalents:

**INNER JOIN = set intersection**  
**OUTER JOIN = set union**  
**RIGHT/LEFT JOIN = union with intersection**

More details can be found at https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins.

Here we will only demonstrate the use of INNER JOIN in two cases:

In [57]:
print('ID, Fname, Lname, Year, s_ID, m_ID')
c.execute("SELECT *                               \
          FROM Actors a                           \
          INNER JOIN starred_in s                 \
          ON a.actor_ID=s.actor_ID                \
          ;" ) 
data = c.fetchall()
for row in data: print(row)

ID, Fname, Lname, Year, s_ID, m_ID
(1, 'Emma', 'Watson', 1990, 1, 2)
(1, 'Emma', 'Watson', 1990, 1, 1)
(2, 'Matt', 'Damon', 1970, 2, 4)


Notice that the quary reports the first 4 columns from the Actors table, and the last 2 columns from the starred_in table. Therefore here we have joined the "Actors" and "starred_in" tables linking them with the "ON" statement that specifies which columns have to be compared for linking. 

As a result we found the intersection of the two tables, in fact, we didn't get the name of any other actor because in the current DB they are not linked to any movie.

#### Crossing several relationship levels

If we want to select actors that star in a certain movie (and we don't know the movie_ID off the top of our head) we have to reach across one more level to make this connection. Please notice how Actors and Movies are not directly linked in the ERD:

<img src="erd_cinema_highlight.png" style="width: 500px;"/>

This operation can be done by nesting as many JOIN statements as the links that have to be crossed in the ERD. In this example we will start from the Actors and select only those that intersect with a set made of the intersection between the "starred_in" table and the "Movies" set where our condition on the title is set.  

So, let's SELECT the Actors that star into the movie "the beauty and the beast":

In [58]:
c.execute("SELECT * FROM Movies m INNER JOIN (         \
          SELECT movie_ID FROM starred_in s            \
          INNER JOIN Actors a ON a.actor_ID=s.actor_ID \
          WHERE a.actor_Lname = 'Watson'\
          ) sm ON m.movie_ID=sm.movie_ID;" ) 
data = c.fetchall()
for row in data: print(row)

(2, 'the beauty and the beast', None, 2017, 1, 'fantasy', None, 2)
(1, 'the circle', None, 2017, 1, 'sci-fi', None, 1)


It will look cryptic for the first 10 minutes, and it's OK. To help you crack it, notice how the first INNER JOIN intersects the Actors with a set created by the query itself and contained between "()". This intermediate set is created with another SELECT statement that contains a nested INNER JOIN between the "starred_in" and the "Movies" table.

#### How to add an extra column

In [ ]:
c.execute("ALTER TABLE Movies ADD COLUMN comment VARCHAR;" )          
# In theory I would use a BOOLEAN type but SQLite doesn't have it
conn.commit()

In [3]:
c.execute('PRAGMA table_info(Movies);')
data=c.fetchall()
data

[(0, 'movie_ID', 'INTEGER', 0, None, 1),
 (1, 'title_en', 'VARCHAR', 0, None, 0),
 (2, 'title_IT', 'VARCHAR', 0, None, 0),
 (3, 'movie_year', 'INT', 0, None, 0),
 (4, 'watched', 'INT', 0, None, 0),
 (5, 'genre', 'VARCHAR', 0, None, 0),
 (6, 'comment', 'VARCHAR', 0, None, 0)]

**More population quaryies**

In [8]:
# Multiple MOVIES
c.execute("INSERT INTO Movies (title_en, movie_year, watched, genre) \
          VALUES ('jurassic park',),                  \
                 ('bicentennial man',),   \
                 ('the lord of the rings',),         \
                 ('batman and robin', ),                  \
                 ('gattaca', ),                \
                 ('the avengers', ),                    \
                 ('going clear', ),        \
                 ('grease',),                \
                 ('inglorious basterds', ),     \
                 ('django unchained', ),    \
                 ('iron man', ),         \
                 ('armageddon',)           \
                 ('the sixth sense', )                        \
                 ('sleepy hollow', )                      \
                 ('batman returns',)                   \
                 ('titanic', )        \
                 ('the man in the iron mask', )      \
                 ('the iviator', )                 \
                 ('inception', )                  \
                 ('the revenant', )                \
                 ('one flew over the cuckoo's nest', )      \
                 ('the shining', )             \
                 ('the bucket list', )        \
                 ('the shawshank redemption', )    \
                 ('bruce almighty',)            \
                 ('batman begins', )          \
                 ('invictus', )            \
                 ('oblivion',)            \
                 ('midnight in paris', ),       \
                 ('the day after tomorrow', )   \
                 ('harry potter', )             \
                 ('schindler's list', )          \
                 ('independence day', )          \
                 ('man in black', )               \
                 ('I robot', )               \
                 ('hitch',)             \
                 ('the pursuit of happiness',)     \
                 ('seven pounds',)             \
                 ('sherlock holmes', )          \
                 ('hook', )                    \
                 ('hidden figures', 2016, 1, 'historic')  \
                 
                 
          ;")
conn.commit()   # insert multiple movies with a single query

In [9]:
# Multiple ACTORS
c.execute("INSERT INTO Actors (actor_Fname, actor_Lname, actor_birthYear) \
          VALUES ('Geoffrey', 'Rush', 1951),                             \
                 ('Sam', 'Neill', 1947),                           \
                 ('Miranda', 'Otto', 1967),                \
                 ('Uma', 'Thurmann', 1970),                                \
                 ('John', 'Travolta', 1954),         \
                 ('Samuel', 'Jackson', 1948),           \
                 ('Christof', 'Walts', 1956),           \
                 ('Bruce', 'Willis', 1955),             \
                 ('Christopher', 'Walken', 1943),       \
                 ('Quentin', 'Tarantino', 1963),        \
                 ('Leonardo', 'Di Caprio', 1974),    \
                 ('Jack', 'Nicholson', 1937),         \
                 ('Morgan', 'Freeman', 1937),           \
                 ('Tom', 'Hanks', 1956),          \
                 ('Owen', 'Wilson', 1968),            \
                 ('Cameron', 'Diaz', 1972),          \
                 ('Jim', 'Carrey', 1962),          \
                 ('Jim', 'Broadbent', 1949),    \
                 ('Liam', 'Neeson', 1952),      \
                 ('Will', 'Smith', 1968),            \
                 ('Robert', 'Downey', 1965),          \
                 ('Kirsten', 'Dunst', 1982),         \
                 ('Chris', 'Evans', 1981),          \
                 ('Mark', 'Ruffalo', 1967),        \
                 ('Scarlett', 'Johannson', 1984),   \
                 ('Gwynett', 'Paltrow', 1972),     \
                 ('Liv', 'Tyler', 1977)        \
          ;")
conn.commit()   # insert multiple movies with a single query

In [65]:
# MOVIE ACTOR RELATIONSHIP
c.execute("INSERT INTO starred_in (actor_ID, movie_ID)   \
          SELECT actor_ID, movie_ID                      \
          FROM Actors, Movies                            \
          WHERE actor_Lname = 'Corbett'                     \
          AND title_IT LIKE 'il mio grosso%';     \
          ")
conn.commit()

In [2]:
c.execute("INSERT INTO Directors (director_Fname, director_Lname)  \
          VALUES ('Quentin', 'Tarantino'),                         \
          ('Martin', 'Scorsese'),                                  \
          ('Steven', 'Spielberg'),                                 \
          ('Christopher', 'Nolan'),                                \
          ('Stanley', 'Kybrick'),                                  \
          ('Alfred', 'Hitchcock'),                                 \
          ('Woody', 'Allen')                                       \
          ")
conn.commit()

In [7]:
# MOVIE-director RELATIONSHIP
c.execute("INSERT INTO directed_by (director_ID, movie_ID)   \
          SELECT director_ID, movie_ID                      \
          FROM Directors, Movies                            \
          WHERE director_Lname = 'Spielberg'                     \
          AND title_en LIKE 'save%';     \
          ")
conn.commit()

In [31]:
c.execute("INSERT OR IGNORE INTO Movies (title_en, movie_year, watched, genre) \
          VALUES ('jurassic park', 1993, 1, 'sci-fi');")
conn.commit()

Here is how to delete entries:

In [29]:
c.execute("DELETE FROM Movies WHERE title_en LIKE 'jurassic%';")
conn.commit()

Here is how to check how many rows have been affected by the last changes, and check if an INSERT or DELECT, ecc has been successful.

In [30]:
c.execute("SELECT changes();")
result = c.fetchall()
print(result)

[(2,)]


## 6. Let's automate the process a bit

Now this is great and fun, but every query is a lot of writing, and we might want to repeat the same or a similar quary for many different values, both in data entry and in data analysis. So what we have to do is to create customized functions that will "edit" the query for us. And eventually programmatically perform data entry from verious structured or unstructured lists. 

First of all we need to start by creating low level functions that perform basic operations, like inserting in a single value in a single table from structured variables, and from there create higher level functions that perform more flexible operations. 

In [66]:
def singleTable_entry(table, column, data):
    """This function formats the query and commits an entry to DB
    
    Parameters:
        table = 'table_name' as a string
        column = ['colum1', 'column2',..] is a list of strings
        data = [123, 'name', 456, ...] is a list of strings and integers
        
    Returns:
        "str" with SQLite command to insert DATA into COLUMN
    """
    
    for i in range(len(data)):
        
        # check for datatype and format accordingly
        if isinstance(data[i], str):
            data[i] = str("'" + data[i] + "'")
            
        elif isinstance(data[i], int):
            data[i] = str(data[i])
    
    # join list elements into one string
    column = ', '.join(column)
    data = ', '.join(data)
    
    #create command string
    cmd = 'INSERT INTO %s (%s) VALUES(%s);' %(table, column, data)
    
    return cmd

with this function we can get the query already fromatted starting from structured data like so:

In [67]:
# some structured data
table = 'my_table'
column = ['my_column1', 'my_column2', 'my_column3', 'my_column3']
value = ['bablabal', 123456, 'blabla2', 58]

#let's use the function on this data
singleTable_entry(table, column, value)

"INSERT INTO my_table (my_column1, my_column2, my_column3, my_column3) VALUES('bablabal', 123456, 'blabla2', 58);"

Notice that the function has been written to recognize two datatypes (str, int) and to format the query to respect that.

In [1]:
import sqlite3

conn = sqlite3.connect('cinema.db', timeout = 10) 
c = conn.cursor()

In [46]:
c.close()
conn.close()